Prerequisite installs for Project 1 - Quantum state tomography (Simulation)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]

Project 1 - Quantum state tomography (Simulation)

In [ ]:
from math                 import sqrt
from qiskit               import QuantumCircuit
from qiskit.primitives    import StatevectorSampler
from qiskit.visualization import plot_bloch_vector


def quantum_state_tomography(alpha, beta):
    """
    Perform quantum state tomography to calculate the Bloch vector 
    of a quantum state defined by alpha and beta.

    Parameters:
    - alpha (float): Amplitude of the |0⟩ basis state.
    - beta (float): Amplitude of the |1⟩ basis state.

    Returns:
    - list: Bloch vector [x, y, z] representing the quantum state.
    """
    # Initialize the quantum circuit with 1 qubit and 1 classical bit
    qc = QuantumCircuit(1, 1)
    
    # Define the target quantum state vector
    target_state_vector = [alpha, beta]

    # Initialize the qubit to the target state
    qc.initialize(target_state_vector, 0)

    # Define the measurement circuit for the X basis
    measure_x = QuantumCircuit(1, 1)
    measure_x.h(0)          # Apply Hadamard gate to switch to X basis
    measure_x.measure(0, 0) # Measure on qubit 0

    # Define the measurement circuit for the Y basis
    measure_y = QuantumCircuit(1, 1)
    measure_y.sdg(0)  # Apply S† gate to account for phase shift
    measure_y.h(0)    # Apply Hadamard gate to switch to Y basis
    measure_y.measure(0, 0) # Measure on qubit 0

    # Define the measurement circuit for the Z basis
    measure_z = QuantumCircuit(1, 1)
    measure_z.measure(0, 0) # Directly measure in the Z basis

    # Combine the initialization circuit with each measurement circuit
    qc_x = qc.compose(measure_x)  # For X-basis measurement
    qc_y = qc.compose(measure_y)  # For Y-basis measurement
    qc_z = qc.compose(measure_z)  # For Z-basis measurement

    # Initialize the state vector sampler
    state_vector_sampler = StatevectorSampler()
    shots = 8192  # Number of shots for each measurement simulation

    # Simulate the measurements in the X, Y, and Z bases
    results_x = state_vector_sampler.run([qc_x], shots=shots).result()
    results_y = state_vector_sampler.run([qc_y], shots=shots).result()
    results_z = state_vector_sampler.run([qc_z], shots=shots).result()

    # Extract measurement results as counts for each basis
    counts_x = results_x[0].data.c.get_counts()  # Counts for X basis
    counts_y = results_y[0].data.c.get_counts()  # Counts for Y basis
    counts_z = results_z[0].data.c.get_counts()  # Counts for Z basis

    # Calculate the Bloch vector components
    # Absolute difference between |0⟩ and |1⟩ counts, normalized by shots
    x = abs(counts_x.get('0', 0) - counts_x.get('1', 0)) / shots
    y = abs(counts_y.get('0', 0) - counts_y.get('1', 0)) / shots
    z = abs(counts_z.get('0', 0) - counts_z.get('1', 0)) / shots

    # Return the Bloch vector as [x, y, z]
    return [x, y, z]

# Define the alpha and beta coefficients for the quantum state
alpha = sqrt(2 / 3) # Amplitude of |0⟩ state
beta  = sqrt(1 / 3) # Amplitude of |1⟩ state

# Perform quantum state tomography and calculate the Bloch vector
bloch_vector = quantum_state_tomography(alpha, beta)

# Print theoretical Bloch vector for comparison
print("Theoretical Bloch vector result:", [(2 * sqrt(2)) / 3, 0, 1 / 3])

# Print the resulting Bloch vector from tomography
print("Resulting Bloch vector:", bloch_vector)

# Visualize the Bloch vector
plot_bloch_vector(bloch_vector)

Prerequisite installs for Project 1 - Quantum state tomography (IBM Quantum Platform)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]
%pip install qiskit-ibm-runtime

Save account to Qiskit, according to configuration file 'ibm_quantum_config.json'.

To get your IBM Quantum Platform token, go to https://quantum.ibm.com/account and copy your API token.
The 'ibm_quantum_config.json' file should contain something like this:

{
    "channel": "ibm_quantum",
    "token": "yourtoken"
}

In [ ]:
import os
import sys
import json
from   qiskit_ibm_runtime import QiskitRuntimeService


if not os.path.exists("ibm_quantum_config.json"):
    print("Please create a file named 'ibm_quantum_config.json' with your IBM Quantum API token.")
    sys.exit(1)

with open("ibm_quantum_config.json", "r") as config_file:
    config = json.load(config_file)
    print("Configuration loaded successfully.")

QiskitRuntimeService.save_account(
    channel=config["channel"],
    token=config["token"],
    overwrite=True
)

print("Account saved successfully.")

Load account from Qiskit.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
print("Account loaded successfully!")

backends = service.backends(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Available Backends: {backends}")

least_busy_backend = service.least_busy(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Least busy backend: {least_busy_backend.name}")

least_busy_backend_nb_qubits = least_busy_backend.configuration().n_qubits
print(f"Least busy backend number of qubits: {least_busy_backend_nb_qubits}")

Project 1 - Quantum state tomography (IBM Quantum Platform)

In [ ]:
from math                 import sqrt
from qiskit               import QuantumCircuit, \
                                 transpile
from qiskit.visualization import plot_bloch_vector
from qiskit_ibm_runtime   import QiskitRuntimeService, \
                                 Session,              \
                                 Sampler

def quantum_state_tomography(alpha, beta):
    """
    Perform quantum state tomography to calculate the Bloch vector 
    of a quantum state defined by alpha and beta using IBM Quantum backends.

    Parameters:
    - alpha (float): Amplitude of the |0⟩ basis state.
    - beta (float): Amplitude of the |1⟩ basis state.

    Returns:
    - list: Bloch vector [x, y, z] representing the quantum state.
    """
    # Initialize Bloch vector components to 0
    x, y, z = 0.0, 0.0, 0.0

    # Create a single-qubit circuit with one classical bit for measurement
    qc = QuantumCircuit(1, 1)

    # Define the target quantum state vector
    target_state_vector = [alpha, beta]

    # Initialize the qubit to the target quantum state
    qc.initialize(target_state_vector, 0)

    # Define the measurement circuit for the X basis
    measure_x = QuantumCircuit(1, 1)
    measure_x.h(0)          # Apply Hadamard gate to switch to the X basis
    measure_x.measure(0, 0) # Measure on qubit 0

    # Define the measurement circuit for the Y basis
    measure_y = QuantumCircuit(1, 1)
    measure_y.sdg(0)        # Apply S† gate to adjust the phase
    measure_y.h(0)          # Apply Hadamard gate to switch to the Y basis
    measure_y.measure(0, 0) # Measure on qubit 0

    # Define the measurement circuit for the Z basis
    measure_z = QuantumCircuit(1, 1)
    measure_z.measure(0, 0) # Directly measure the qubit in the Z basis

    # Combine the initialization circuit with the X, Y, and Z measurement circuits
    qc_x = qc.compose(measure_x)
    qc_y = qc.compose(measure_y)
    qc_z = qc.compose(measure_z)

    # Load the IBM Quantum account for accessing backend services
    service = QiskitRuntimeService()
    print("Account loaded successfully!")

    # Select the least busy backend that is not a simulator and is operational
    least_busy_backend = service.least_busy(
        filters=lambda b: not b.configuration().simulator and b.status().operational
    )
    print(f"Running on backend: {least_busy_backend.name}")

    # Transpile the combined circuits for execution on the selected backend
    qc_x_transpiled = transpile(qc_x, least_busy_backend)
    qc_y_transpiled = transpile(qc_y, least_busy_backend)
    qc_z_transpiled = transpile(qc_z, least_busy_backend)

    # Execute the circuits using the IBM Quantum Runtime session
    with Session(backend=least_busy_backend) as session:
        sampler = Sampler(session=session) # Create a sampler object for the session

        # Specify the number of shots (repetitions of the circuit execution)
        shots = 8192

        # Submit jobs for the transpiled circuits and collect results
        job_qc_x = sampler.run([qc_x_transpiled], shots=shots)
        job_qc_y = sampler.run([qc_y_transpiled], shots=shots)
        job_qc_z = sampler.run([qc_z_transpiled], shots=shots)

        # Retrieve results for each circuit
        job_qc_x_result = job_qc_x.result()
        job_qc_y_result = job_qc_y.result()
        job_qc_z_result = job_qc_z.result()

        # Extract the measurement counts for each basis
        counts_x = job_qc_x_result[0].data.c.get_counts()  # X-basis counts
        counts_y = job_qc_y_result[0].data.c.get_counts()  # Y-basis counts
        counts_z = job_qc_z_result[0].data.c.get_counts()  # Z-basis counts

        # Calculate the Bloch vector components
        # Use the normalized difference between counts for |0⟩ and |1⟩
        x = abs(counts_x.get('0', 0) - counts_x.get('1', 0)) / shots
        y = abs(counts_y.get('0', 0) - counts_y.get('1', 0)) / shots
        z = abs(counts_z.get('0', 0) - counts_z.get('1', 0)) / shots

    # Return the Bloch vector [x, y, z]
    return [x, y, z]

# Define the coefficients for the quantum state |ψ⟩ = α|0⟩ + β|1⟩
alpha = sqrt(2 / 3) # Amplitude of |0⟩ state
beta  = sqrt(1 / 3) # Amplitude of |1⟩ state

# Perform quantum state tomography to calculate the Bloch vector
bloch_vector = quantum_state_tomography(alpha, beta)

# Print the theoretical Bloch vector for comparison
print("Theoretical Bloch vector result:", [(2 * sqrt(2)) / 3, 0, 1 / 3])

# Print the Bloch vector calculated from tomography
print("Resulting Bloch vector:", bloch_vector)

# Visualize the Bloch vector using the Qiskit visualization tool
plot_bloch_vector(bloch_vector)